In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved
# Licensed under the MIT License.
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import os

import numpy as np

import shapely.geometry

from pigeon import annotate

from temporal_cluster_matching import utils, DataInterface, algorithms

## NAIP / Poultry barn labeling

In [ ]:
OUTPUT_FN = "../results/poultry_barn_labels2.csv"

In [ ]:
geoms = utils.get_poultry_barn_geoms("../data/")
len(geoms)

In [ ]:
dataloader = DataInterface.NAIPDataLoader()

In [ ]:
used_idxs = set()
if os.path.exists(OUTPUT_FN):
    with open(OUTPUT_FN, "r") as f:
        lines = f.read().strip().split("\n")
        for i in range(1, len(lines)):
            parts = lines[i].split(",")
            idx = int(parts[0])
            used_idxs.add(idx)
available_idxs = list(set(range(len(geoms))) - used_idxs)

print("Already labeled %d idxs, choosing from %d remaining" % (len(used_idxs), len(available_idxs)))

num_to_label = 5
idxs = np.random.choice(available_idxs, size=num_to_label, replace=False)

In [ ]:
def show_idx(i):
    print("Labeling %d" % (i))
    shape_area = shapely.geometry.shape(geoms[i]).area
    side_length = np.sqrt(shape_area)

    buffer_size = max(side_length * 2, 0.001)

    rgb_images, years = dataloader.get_rgb_stack_from_geom(geoms[i], buffer=buffer_size)
    years = [
        "%d; %d" % (j, year)
        for j, year in enumerate(years)
    ]
    utils.show_images(rgb_images, years)
    
annotations = annotate(
    idxs,
    options=[str(i) for i in range(5)],
    display_fn=lambda i: show_idx(i)
)

In [ ]:
if not os.path.exists(OUTPUT_FN):
    with open(OUTPUT_FN, "w") as f:
        f.write("idx,year\n")
    
with open(OUTPUT_FN, "a") as f:
    for idx, label in annotations:
        years = dataloader.get_years_from_geom(geoms[idx])
        label = int(label)
        year = years[label]
        f.write("%d,%d\n" % (idx, year))

## Sentinel 2 / Solar farm labeling

In [ ]:
OUTPUT_FN = "../results/solar_farm_labels.csv"

In [ ]:
geoms = utils.get_solar_farm_geoms("../data/")
len(geoms)

In [ ]:
dataloader = DataInterface.S2DataLoader()

In [ ]:
used_idxs = set()
if os.path.exists(OUTPUT_FN):
    with open(OUTPUT_FN, "r") as f:
        lines = f.read().strip().split("\n")
        for i in range(1, len(lines)):
            parts = lines[i].split(",")
            idx = int(parts[0])
            used_idxs.add(idx)
available_idxs = list(set(range(len(geoms))) - used_idxs)

print("Already labeled %d idxs, choosing from %d remaining" % (len(used_idxs), len(available_idxs)))

num_to_label = 50
idxs = np.random.choice(available_idxs, size=num_to_label, replace=False)

In [ ]:
idxs = range(935)

In [ ]:
images = []
for i in idxs:
    if i % 50 == 0:
        print(i, len(idxs))
    rgb_images, years = dataloader.get_rgb_stack_from_geom(geoms[i], buffer=0.002)
    images.append(rgb_images)

In [ ]:
titles = [
    "%d, %d" % (i + 2016, i)
    for i in range(5)
]

In [ ]:
def show_idx(i):
    print("Labeling %d" % (i))
    utils.show_images(images[i], titles)
annotations = annotate(
    idxs,
    options=[str(i) for i in range(5)],
    display_fn=lambda i: show_idx(i)
)

In [ ]:
if not os.path.exists(OUTPUT_FN):
    with open(OUTPUT_FN, "w") as f:
        f.write("idx,year\n")
    
with open(OUTPUT_FN, "a") as f:
    for idx, label in annotations:
        years = dataloader.get_years_from_geom(geoms[idx])
        label = int(label)
        year = years[label]
        f.write("%d,%d\n" % (idx, year))